In [1]:
import os
os.environ["SPARK_HOME"] = "/opt/spark"

import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName="bda-spark")

In [2]:
# Check if Spark is working with a little pi calculation with monte carlo
import random
num_samples = 100000
def inside(p):     
  x, y = random.random(), random.random()
  return x*x + y*y < 1
count = sc.parallelize(range(0, num_samples)).filter(inside).count()
pi = 4 * count / num_samples
print(pi)

3.14604


In [68]:
rdd_list = []
path_trip = "datasets/trip_data/"
path_fair = "datasets/trip_fare/"
for e in os.listdir(path_trip):
    rdd_list.append(sc.textFile(path_trip+e))
len(rdd_list)

12

In [34]:
data1 = sc.textFile('datasets/trip_data/trip_data_9.csv')

In [32]:
rdd_list[0]

datasets/trip_data/trip_data_9.csv MapPartitionsRDD[48] at textFile at <unknown>:0

In [8]:
rdd_list[0].count()

14107694

In [38]:
rdd_list[0].take(2)

['medallion, hack_license, vendor_id, rate_code, store_and_fwd_flag, pickup_datetime, dropoff_datetime, passenger_count, trip_time_in_secs, trip_distance, pickup_longitude, pickup_latitude, dropoff_longitude, dropoff_latitude',
 '0CF8B9F42FED24FA1CA8AACA36D1A25B,A04B37232EB2478C7A831A6C587C15B4,CMT,1,N,2013-09-01 16:35:05,2013-09-01 16:47:53,2,767,2.60,-73.9879,40.724079,-73.994598,40.75058']

In [61]:
rdd_list[0].first()

'medallion, hack_license, vendor_id, rate_code, store_and_fwd_flag, pickup_datetime, dropoff_datetime, passenger_count, trip_time_in_secs, trip_distance, pickup_longitude, pickup_latitude, dropoff_longitude, dropoff_latitude'

In [3]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [4]:
df = (sqlContext.read.format("csv")
      .option("delimiter", ",")
      .option("header", "true")
      .load("datasets/trip_data/trip_data_9.csv")
      .drop("medallion"," hack_license")
      .withColumnRenamed(" vendor_id", "vendor_id")
      .withColumnRenamed(" rate_code", "rate_code")
      .withColumnRenamed(" store_and_fwd_flag", "store_and_fwd_flag")
      .withColumnRenamed(" pickup_datetime", "pickup_datetime")
      .withColumnRenamed(" dropoff_datetime", "dropoff_datetime")
      .withColumnRenamed(" passenger_count", "passenger_count")
      .withColumnRenamed(" trip_time_in_secs", "trip_time_in_secs")
      .withColumnRenamed(" trip_distance", "trip_distance")
      .withColumnRenamed(" pickup_longitude", "pickup_longitude")
      .withColumnRenamed(" pickup_latitude", "pickup_latitude")
      .withColumnRenamed(" dropoff_longitude", "dropoff_longitude")
      .withColumnRenamed(" dropoff_latitude", "dropoff_latitude")
     )
df.count()
#df.describe().show()

14107693

In [67]:
df = (df.withColumn('pickup_longitude', df['pickup_longitude'].cast('float')) #convert str to float
      .withColumn('pickup_latitude', df['pickup_latitude'].cast('float'))
      .withColumn('dropoff_longitude', df['dropoff_longitude'].cast('float'))
      .withColumn('dropoff_latitude', df['dropoff_latitude'].cast('float'))
      .withColumn('trip_distance', df['trip_distance'].cast('float'))
      .na.drop() #removes rows with null
     )
df.count()

6919647

In [58]:
df.head(1)

[Row(vendor_id='CMT', rate_code='1', store_and_fwd_flag='N', pickup_datetime='2013-09-01 16:35:05', dropoff_datetime='2013-09-01 16:47:53', passenger_count='2', trip_time_in_secs='767', trip_distance=2.5999999046325684, pickup_longitude=-73.98789978027344, pickup_latitude=40.72407913208008, dropoff_longitude=-73.99459838867188, dropoff_latitude=40.750579833984375)]

In [31]:
df.columns

['vendor_id',
 'rate_code',
 'store_and_fwd_flag',
 'pickup_datetime',
 'dropoff_datetime',
 'passenger_count',
 'trip_time_in_secs',
 'trip_distance',
 'pickup_longitude',
 'pickup_latitude',
 'dropoff_longitude',
 'dropoff_latitude']

In [51]:
df2.head(3)

[Row( hack_license='A04B37232EB2478C7A831A6C587C15B4',  vendor_id='CMT',  rate_code='1',  store_and_fwd_flag='N',  pickup_datetime='2013-09-01 16:35:05',  dropoff_datetime='2013-09-01 16:47:53',  passenger_count='2',  trip_time_in_secs='767',  trip_distance='2.60',  pickup_longitude='-73.9879',  pickup_latitude='40.724079',  dropoff_longitude='-73.994598',  dropoff_latitude='40.75058'),
 Row( hack_license='8C1A6E14ED1D019FBD227970CC619496',  vendor_id='CMT',  rate_code='1',  store_and_fwd_flag='N',  pickup_datetime='2013-09-01 17:44:05',  dropoff_datetime='2013-09-01 17:58:37',  passenger_count='1',  trip_time_in_secs='871',  trip_distance='3.50',  pickup_longitude='-74.007866',  pickup_latitude='40.710232',  dropoff_longitude='-74.003975',  dropoff_latitude='40.756569'),
 Row( hack_license='96F7564919171F55F551F0F9B96B5199',  vendor_id='CMT',  rate_code='1',  store_and_fwd_flag='N',  pickup_datetime='2013-09-01 16:36:20',  dropoff_datetime='2013-09-01 16:50:53',  passenger_count='1', 

In [75]:
df.show()

+---------+---------+------------------+-------------------+-------------------+---------------+-----------------+-------------+----------------+---------------+-----------------+----------------+
|vendor_id|rate_code|store_and_fwd_flag|    pickup_datetime|   dropoff_datetime|passenger_count|trip_time_in_secs|trip_distance|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|
+---------+---------+------------------+-------------------+-------------------+---------------+-----------------+-------------+----------------+---------------+-----------------+----------------+
|      CMT|        1|                 N|2013-09-01 16:35:05|2013-09-01 16:47:53|              2|              767|          2.6|        -73.9879|       40.72408|         -73.9946|        40.75058|
|      CMT|        1|                 N|2013-09-01 17:44:05|2013-09-01 17:58:37|              1|              871|          3.5|      -74.007866|       40.71023|       -74.003975|        40.75657|
|      CMT|    

In [5]:
def df_maker(file_path):
    print(file_path)
    return (sqlContext.read.format("csv")
                .option("delimiter", ",")
                .option("header", "true")
                .load(file_path)
                .drop("medallion")
                .drop(" hack_license")
                .drop("hack_license")
                .withColumnRenamed(" vendor_id", "vendor_id")
                .withColumnRenamed(" rate_code", "rate_code")
                .withColumnRenamed(" store_and_fwd_flag", "store_and_fwd_flag")
                .withColumnRenamed(" pickup_datetime", "pickup_datetime")
                .withColumnRenamed(" dropoff_datetime", "dropoff_datetime")
                .withColumnRenamed(" passenger_count", "passenger_count")
                .withColumnRenamed(" trip_time_in_secs", "trip_time_in_secs")
                .withColumnRenamed(" trip_distance", "trip_distance")
                .withColumnRenamed(" pickup_longitude", "pickup_longitude")
                .withColumnRenamed(" pickup_latitude", "pickup_latitude")
                .withColumnRenamed(" dropoff_longitude", "dropoff_longitude")
                .withColumnRenamed(" dropoff_latitude", "dropoff_latitude")
           )

In [6]:
def df_filter(df):
    return (df.withColumn('pickup_longitude', df['pickup_longitude'].cast('float')) #convert str to float
          .withColumn('pickup_latitude', df['pickup_latitude'].cast('float'))
          .withColumn('dropoff_longitude', df['dropoff_longitude'].cast('float'))
          .withColumn('dropoff_latitude', df['dropoff_latitude'].cast('float'))
          .withColumn('trip_distance', df['trip_distance'].cast('float'))
          .filter(df.trip_distance<20.0) #filter out long distance, miles
          .na.drop() #removes rows with null
         )

In [7]:
path_trip = "datasets/trip_data/"
for idx,e in enumerate(os.listdir(path_trip)):
    if idx == 0:
        print("in 0",e)
        dff = df_filter(df_maker(path_trip+e))
        #break
    else:
        #print(e)
        df = df_filter(df_maker(path_trip+e))
        print(df.columns)
        dff=dff.union(df)
        #break
    #print(dff.count()) #to comment in prod, very time consuming
dff.show()

in 0 trip_data_9.csv
datasets/trip_data/trip_data_9.csv
datasets/trip_data/trip_data_2.csv
['vendor_id', 'rate_code', 'store_and_fwd_flag', 'pickup_datetime', 'dropoff_datetime', 'passenger_count', 'trip_time_in_secs', 'trip_distance', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude']
datasets/trip_data/trip_data_11.csv
['vendor_id', 'rate_code', 'store_and_fwd_flag', 'pickup_datetime', 'dropoff_datetime', 'passenger_count', 'trip_time_in_secs', 'trip_distance', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude']
datasets/trip_data/trip_data_7.csv
['vendor_id', 'rate_code', 'store_and_fwd_flag', 'pickup_datetime', 'dropoff_datetime', 'passenger_count', 'trip_time_in_secs', 'trip_distance', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude']
datasets/trip_data/trip_data_5.csv
['vendor_id', 'rate_code', 'store_and_fwd_flag', 'pickup_datetime', 'dropoff_datetime', 'passenger_count', 'trip_time_in_secs', 

In [97]:
dff.count()

86322541

In [103]:
df.count()

7901193

In [8]:
dff_copy = dff
df_copy = df

In [9]:
## Fititng K means Model 
from pyspark.ml.feature import VectorAssembler
vecAssembler = VectorAssembler(inputCols=["dropoff_latitude", "dropoff_longitude"], outputCol="features")
df_features_dropoff = vecAssembler.transform(df_copy) #DF COPY HERE, SMALLER THAN DFF

In [10]:
from pyspark.ml.clustering import KMeans

kmeans = KMeans(k=15, seed=1) 
model = kmeans.fit(df_features_dropoff.select('features'))

In [11]:
### Vecorizing and getting pickup clusters
vecAssembler = VectorAssembler(inputCols=["pickup_latitude", "pickup_longitude"], outputCol="features")
df_features_pickup = vecAssembler.transform(df_copy)
df_copy = model.transform(df_features_pickup)

In [12]:
## assigniung prediction to pickup cluster
df_copy = df_copy.withColumnRenamed('prediction', 'pickup_cluster')
df_copy = df_copy.drop('features')

In [13]:
### Vecorizing and getting dropoff clusters
vecAssembler = VectorAssembler(inputCols=["dropoff_latitude", "dropoff_longitude"], outputCol="features")
df_features_dropoff = vecAssembler.transform(df_copy)
df_copy = model.transform(df_features_dropoff)

In [14]:
## assigniung prediction to dropoff cluster
df_copy = df_copy.withColumnRenamed('prediction', 'dropoff_cluster')
df_copy = df_copy.drop('features')

## Let's plot it
### First step is to convert it to Panda Dataframe

In [ ]:
pd_df = df_copy.toPandas()

In [ ]:
pd_df = pd_df.sample(frac= 0.1)

### Cluster it with Seaborn

In [ ]:
import seaborn as sns
%matplotlib inline

sns.set_style("whitegrid")
sns.lmplot(x="pickup_latitude", y="pickup_longitude",data = pd_df[pd_df['pickup_latitude']!=0.0],fit_reg=False,hue='pickup_cluster',size=10,scatter_kws={"s":100})

In [ ]:
sc.stop()